In [5]:
from sqlite3 import Date
import pandas as pd
import yfinance as yf
import datetime as datetime
from prophet import Prophet
from time import strftime
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel

In [6]:
ticker = 'AAPL'
today = datetime.date.today()
end_date = today.strftime("%Y-%m-%d")
data = yf.Ticker(ticker).history(period='max', interval='1d')
data

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1980-12-12 00:00:00-05:00,0.099192,0.099623,0.099192,0.099192,469033600,0.0,0.0
1980-12-15 00:00:00-05:00,0.094448,0.094448,0.094017,0.094017,175884800,0.0,0.0
1980-12-16 00:00:00-05:00,0.087548,0.087548,0.087117,0.087117,105728000,0.0,0.0
1980-12-17 00:00:00-05:00,0.089273,0.089704,0.089273,0.089273,86441600,0.0,0.0
1980-12-18 00:00:00-05:00,0.091861,0.092292,0.091861,0.091861,73449600,0.0,0.0
...,...,...,...,...,...,...,...
2024-04-12 00:00:00-04:00,174.259995,178.360001,174.210007,176.550003,101593300,0.0,0.0
2024-04-15 00:00:00-04:00,175.360001,176.630005,172.500000,172.690002,73531800,0.0,0.0
2024-04-16 00:00:00-04:00,171.750000,173.759995,168.270004,169.380005,73711200,0.0,0.0


In [7]:
df_forecast = data.copy()
df_forecast.reset_index(inplace=True)
df_forecast

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,1980-12-12 00:00:00-05:00,0.099192,0.099623,0.099192,0.099192,469033600,0.0,0.0
1,1980-12-15 00:00:00-05:00,0.094448,0.094448,0.094017,0.094017,175884800,0.0,0.0
2,1980-12-16 00:00:00-05:00,0.087548,0.087548,0.087117,0.087117,105728000,0.0,0.0
3,1980-12-17 00:00:00-05:00,0.089273,0.089704,0.089273,0.089273,86441600,0.0,0.0
4,1980-12-18 00:00:00-05:00,0.091861,0.092292,0.091861,0.091861,73449600,0.0,0.0
...,...,...,...,...,...,...,...,...
10923,2024-04-12 00:00:00-04:00,174.259995,178.360001,174.210007,176.550003,101593300,0.0,0.0
10924,2024-04-15 00:00:00-04:00,175.360001,176.630005,172.500000,172.690002,73531800,0.0,0.0
10925,2024-04-16 00:00:00-04:00,171.750000,173.759995,168.270004,169.380005,73711200,0.0,0.0
10926,2024-04-17 00:00:00-04:00,169.610001,170.649994,168.000000,168.000000,50843400,0.0,0.0


In [8]:
df_forecast["ds"] = df_forecast['Date'].dt.date
df_forecast["y"] = df_forecast["Close"]
df_forecast = df_forecast[["ds","y"]]
df_forecast

,ds,y
0,1980-12-12,0.099192
1,1980-12-15,0.094017
2,1980-12-16,0.087117
3,1980-12-17,0.089273
4,1980-12-18,0.091861
...,...,...
10923,2024-04-12,176.550003
10924,2024-04-15,172.690002
10925,2024-04-16,169.380005
10926,2024-04-17,168.000000


In [9]:
model = Prophet()
model.fit(df_forecast)

22:06:06 - cmdstanpy - INFO - Chain [1] start processing
22:06:09 - cmdstanpy - INFO - Chain [1] done processing


In [10]:
future = pd.to_datetime(end_date) + pd.DateOffset(days=30)
future_date = future.strftime("%Y-%m-%d")
dates = pd.date_range(start= end_date, end= future_date)
df_pred = pd.DataFrame({"ds" : dates})
forecast = model.predict(df_pred)
prediction_list = forecast.tail(30)
prediction_list

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
1,2024-04-19,181.477217,170.587468,191.944377,181.477217,181.477217,-0.360043,-0.360043,-0.360043,-0.064312,-0.064312,-0.064312,-0.295732,-0.295732,-0.295732,0.0,0.0,0.0,181.117173
2,2024-04-20,181.534244,170.639092,192.025676,181.534244,181.534244,-0.289994,-0.289994,-0.289994,-0.004684,-0.004684,-0.004684,-0.285310,-0.285310,-0.285310,0.0,0.0,0.0,181.244251
3,2024-04-21,181.591272,170.318767,192.811932,181.591272,181.591272,-0.281172,-0.281172,-0.281172,-0.004684,-0.004684,-0.004684,-0.276489,-0.276489,-0.276489,0.0,0.0,0.0,181.310100
4,2024-04-22,181.648300,169.877264,192.266286,181.648300,181.648300,-0.267191,-0.267191,-0.267191,0.002302,0.002302,0.002302,-0.269493,-0.269493,-0.269493,0.0,0.0,0.0,181.381108
5,2024-04-23,181.705327,170.343823,191.959176,181.705327,181.705327,-0.241886,-0.241886,-0.241886,0.022647,0.022647,0.022647,-0.264533,-0.264533,-0.264533,0.0,0.0,0.0,181.463442
6,2024-04-24,181.762355,169.881496,192.348119,181.762355,181.762355,-0.211094,-0.211094,-0.211094,0.050700,0.050700,0.050700,-0.261794,-0.261794,-0.261794,0.0,0.0,0.0,181.551261
7,2024-04-25,181.819383,170.692650,192.672972,181.819383,181.819383,-0.263401,-0.263401,-0.263401,-0.001970,-0.001970,-0.001970,-0.261432,-0.261432,-0.261432,0.0,0.0,0.0,181.555981
8,2024-04-26,181.876410,171.326142,193.117028,181.876410,181.876410,-0.327879,-0.327879,-0.327879,-0.064312,-0.064312,-0.064312,-0.263567,-0.263567,-0.263567,0.0,0.0,0.0,181.548531
9,2024-04-27,181.933438,170.296270,192.675717,181.933438,181.933438,-0.272963,-0.272963,-0.272963,-0.004684,-0.004684,-0.004684,-0.268279,-0.268279,-0.268279,0.0,0.0,0.0,181.660475
10,2024-04-28,181.990466,170.775286,192.103438,181.990466,181.990466,-0.280285,-0.280285,-0.280285,-0.004684,-0.004684,-0.004684,-0.275601,-0.275601,-0.275601,0.0,0.0,0.0,181.710181


In [11]:
forecast = prediction_list[['ds','trend']]
forecast

,ds,trend
1,2024-04-19,181.477217
2,2024-04-20,181.534244
3,2024-04-21,181.591272
4,2024-04-22,181.648300
5,2024-04-23,181.705327
6,2024-04-24,181.762355
7,2024-04-25,181.819383
8,2024-04-26,181.876410
9,2024-04-27,181.933438
10,2024-04-28,181.990466


In [12]:
print(forecast.to_json(orient='records'))

[{"ds":1713484800000,"trend":181.4772166681},{"ds":1713571200000,"trend":181.5342443372},{"ds":1713657600000,"trend":181.5912720063},{"ds":1713744000000,"trend":181.6482996754},{"ds":1713830400000,"trend":181.7053273445},{"ds":1713916800000,"trend":181.7623550136},{"ds":1714003200000,"trend":181.8193826828},{"ds":1714089600000,"trend":181.8764103519},{"ds":1714176000000,"trend":181.933438021},{"ds":1714262400000,"trend":181.9904656901},{"ds":1714348800000,"trend":182.0474933592},{"ds":1714435200000,"trend":182.1045210283},{"ds":1714521600000,"trend":182.1615486974},{"ds":1714608000000,"trend":182.2185763666},{"ds":1714694400000,"trend":182.2756040357},{"ds":1714780800000,"trend":182.3326317048},{"ds":1714867200000,"trend":182.3896593739},{"ds":1714953600000,"trend":182.446687043},{"ds":1715040000000,"trend":182.5037147121},{"ds":1715126400000,"trend":182.5607423812},{"ds":1715212800000,"trend":182.6177700504},{"ds":1715299200000,"trend":182.6747977195},{"ds":1715385600000,"trend":182.7

In [13]:
print(forecast.to_json(orient='index'))

{"1":{"ds":1713484800000,"trend":181.4772166681},"2":{"ds":1713571200000,"trend":181.5342443372},"3":{"ds":1713657600000,"trend":181.5912720063},"4":{"ds":1713744000000,"trend":181.6482996754},"5":{"ds":1713830400000,"trend":181.7053273445},"6":{"ds":1713916800000,"trend":181.7623550136},"7":{"ds":1714003200000,"trend":181.8193826828},"8":{"ds":1714089600000,"trend":181.8764103519},"9":{"ds":1714176000000,"trend":181.933438021},"10":{"ds":1714262400000,"trend":181.9904656901},"11":{"ds":1714348800000,"trend":182.0474933592},"12":{"ds":1714435200000,"trend":182.1045210283},"13":{"ds":1714521600000,"trend":182.1615486974},"14":{"ds":1714608000000,"trend":182.2185763666},"15":{"ds":1714694400000,"trend":182.2756040357},"16":{"ds":1714780800000,"trend":182.3326317048},"17":{"ds":1714867200000,"trend":182.3896593739},"18":{"ds":1714953600000,"trend":182.446687043},"19":{"ds":1715040000000,"trend":182.5037147121},"20":{"ds":1715126400000,"trend":182.5607423812},"21":{"ds":1715212800000,"tren